# Транформеры для решения seq2seq задач

Seq2seq - наверное самая общая формальная постановка задачи в NLP. Нужно из произвольной последовательности получить какую-то другую последовательность. И в отличие от разметки последовательности (sequence labelling) не требуется, чтобы обе последовательности совпадали по длине. Даже стандартную задачу классификации можно решать как seq2seq - можно рассматривать метку класса как последовательность длинны 1.

А трансформеры - sota архитектура для seq2seq задач. Мы не будем подробно разбирать устройство транформеров, если вам интересно вы можете поразбираться вот с этими материалами:

Оригинальная статья (сложновато) - https://arxiv.org/pdf/1706.03762.pdf

https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/  
https://jalammar.github.io/illustrated-transformer/

https://www.youtube.com/watch?v=iDulhoQ2pro

https://www.youtube.com/watch?v=TQQlZhbC5ps

Самый известный туториал (на торче) - https://nlp.seas.harvard.edu/2018/04/03/attention.html



Трансформеры будут подробно разбираться на курсе глубокого обучения (по выбору) на втором курсе.

Пока просто попробуем обучать модель на задаче машинного перевода. Для таких задач лучше всего использовать предобученные модели, но если у вас будет какая-то специфичная seq2seq задача, то имеет смысл попробовать обучить трансформер с нуля и в этой тертрадке вам нужно будет поменять только часть с загрузкой данных. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tokenizer_en = Tokenizer.from_file("./torch_weights/tokenizer_en")
tokenizer_ru = Tokenizer.from_file("./torch_weights/tokenizer_ru")

Переводим текст в индексы вот таким образом. В начало добавляем токен '[CLS]', а в конец '[SEP]'. Если вспомните занятие по языковому моделированию, то там мы добавляли "\<start>" и "\<end>" - cls и sep по сути тоже самое. Вы поймете почему именно cls и sep, а не start и end, если подробнее поразбираетесь с устройством трансформеров

In [3]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [183]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')

PAD_IDX

3

In [5]:
# ограничимся длинной в 30 и 35 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_en, max_len_ru = 30, 35

# Код трансформера

Дальше код модели, он взят вот отсюда (с небольшими изменениями) - https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Там есть комментарии по каждому этапу

In [184]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Обратите внимание на то как мы подаем данные в модель

In [185]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
RU_VOCAB_SIZE = tokenizer_ru.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, RU_VOCAB_SIZE, FFN_HID_DIM)
transformer = torch.load("./torch_weights/model")

---
### Homework starts here

Disclaime: Этой домашкой я бы хотел закрыть пропуски в домашках 6, 8, 9. Остальные (скорее всего 10/11/12) я постараюсь досдать
---

In [186]:
from typing import *

In [190]:
def batch_encode(texts: List[str], max_len: int) -> Tuple[Tensor, Tensor]:
    encodings = tokenizer_en.encode_batch(texts)
    encodings = [
        [tokenizer_en.token_to_id('[CLS]')] + encoding.ids[:max_len] + [tokenizer_en.token_to_id('[SEP]')]
        for encoding in encodings
    ]
    outputs = [[tokenizer_ru.token_to_id('[CLS]')]]*len(texts)
    
    input_ids_pad = torch.nn.utils.rnn.pad_sequence(
        [torch.LongTensor(input_ids) for input_ids in encodings],
        batch_first=False,
        padding_value=PAD_IDX
    ).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence(
        [torch.LongTensor(output_ids) for output_ids in outputs],
        batch_first=False,
        padding_value=PAD_IDX
    ).to(DEVICE)
    
    return input_ids_pad, output_ids_pad

In [193]:
SEP_IDX = tokenizer_ru.token_to_id("[SEP]")

In [194]:
def batch_decode(output_ids_pad: Tensor) -> List[str]:
    batch_size = output_ids_pad.shape[1]
    decode = []
    for sequence_idx in range(batch_size):
        sequence = output_ids_pad[:, sequence_idx].cpu().numpy() # to ensure it is on cpu
        filtered_sequence = []
        for token_id in sequence:
            if token_id not in {PAD_IDX, SEP_IDX}:
                filtered_sequence.append(token_id)
            else: # found sep or pad, stopping
                break
                
        decode.append(tokenizer_ru.decode(filtered_sequence))
    
    return decode

In [215]:
def translate(texts: List[str], max_input_len: int = 30, max_output_len: int = 35): # now working with batches!

    input_ids_pad, output_ids_pad = batch_encode(texts, max_input_len)

    (texts_en_mask, texts_ru_mask, 
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                   texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    
    pred = torch.softmax(logits, -1).argmax(-1) # it needs softmaxing
    for i in range(max_output_len):
        output_ids_pad = torch.cat(
            (output_ids_pad, pred)
        )
        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        
        # argmax over last token + unsqueeze to create seq_length dimension
        pred = torch.softmax(logits, -1).argmax(-1)[-1].unsqueeze(0)

    return batch_decode(output_ids_pad)

In [216]:
translate(["Example", "Also another cruel and super-evil megaexample"])

['Пример', 'Еще один жесто кий и супер зло го мега пример']

In [220]:
big_news_text = """
More than half of U.S. states have lowered some barriers to voting
since the 2020 election, making permanent practices that helped
produce record voter turnout during the coronavirus pandemic — a
striking countertrend to the passage this year of restrictions in
key Republican-controlled states.
New laws in states from Vermont to California expand access to the
voting process on a number of fronts, such as offering more options
for early and mail voting, protecting mail ballots from being improperly
rejected and making registering to vote easier.
Some states restored voting rights to people with past felony convictions
or expanded options for voters with disabilities, two long-standing priorities
among voting advocates. And in Virginia, a new law requires localities
to receive preapproval or feedback on voting changes as a shield against
racial discrimination, a first for states after the Supreme Court struck
down a key part of the federal Voting Rights Act in 2013.
Kentucky Secretary of State Michael Adams, a Republican who fought for his
state’s policy changes, said the GOP needs to “stop being scared of voters.”
“Let them vote, and go out and make the case,” he said in an interview,
adding: “I want Republicans to succeed. I think it’s an unforced error to
shoot themselves in the foot in these states by shrinking access. You don’t need to do that.”
Seventy-one new laws easing voting rules are poised to benefit 63 million 
eligible voters in 28 states, or about one-quarter of the U.S. voting population,
according to the Voting Rights Lab report, which tracked policy changes as of June 13.
Thirty-one new laws in 18 states create more barriers to the ballot box,
affecting 36 million eligible voters, or 15 percent of the national voting
population, the report stated.
Legislative debates over restrictions are underway in key states such as Texas
and Pennsylvania, leaving open the possibility that new limitations affecting
millions more voters still will be enacted this year.
"""
## part of text from here: https://www.washingtonpost.com/politics/voting-rights-expansion-states/2021/06/22/1699a6b0-cf87-11eb-8014-2f3926ca24d9_story.html

In [210]:
!pip install -q sentence-splitter

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [221]:
from sentence_splitter import SentenceSplitter
splitter = SentenceSplitter('en')

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        
        
# target function to translate big texts
# works by splitting text into sentences and batched translation of those sentences
def translate_big_text(text, batch_size: int = 8):
    sentences = splitter.split(text)
    translated_sentences = []
    for sentences_batch in batch(sentences, n=batch_size):
        translated_sentences += translate(sentences_batch, 100, 100)
        
    return ".\n".join(translated_sentences)

In [222]:
print(translate_big_text(big_news_text, 8))

Более половины американских государств снизи ли некоторые барье ры для голосования ..
С 2020 года выборы , принятие постоянных практики , которые помогли.
Вы можете получить запись избирателей во время панде мии в панде мии : a.
С мет ровая тенденция к принятию в этом году ограничений в отношении ограничения.
Клю чи республикан ские под контролем ..
Новые законы в шта тах Вер монт и Калифорния расши ряет доступ к.
Процесс голосования по ряду направлений , таких , как предложение больше вариантов.
для раннего и почта голосования , защиту электронной почте в качестве официального утверждения , в отношении того , чтобы они были должным образом.
откло нил и регистри ру ясь на голосование ..
Некоторые государства восстано вили права на голосование в отношении лиц , имеющих прошлое обви нение в убийстве.
или расши ряет варианты избирателей с ограниченными возможностями , два долго стоя тельства.
Среди участников голосования ..
И в Вирджи нии новый закон требует местных особенностей.
до получ